In [1]:
from pylab import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
df = pd.read_csv("ml-100k/u.data",sep="\t",header=None)
df.columns = ["User", "Movie", "Ranking", "Temps"]
df.head()

,User,Movie,Ranking,Temps
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
print df.shape
n_users = len(pd.unique(df.User))
n_movies = len(pd.unique(df.Movie))
print len(pd.unique(df.User))
print len(pd.unique(df.Movie))
print len(pd.unique(df.User))*len(pd.unique(df.Movie))


(100000, 4)
943
1682
1586126


In [178]:
# M = np.zeros([n_users,n_movies])
# for iu,u in enumerate(np.sort(pd.unique(df.User))):
#     for im,m in enumerate(np.sort(pd.unique(df.Movie))):
#         a = (df.User==u).values
#         b = (df.Movie==m).values
#         ix = np.logical_and(a,b)
#         if np.sum(ix)>0:
#             M[iu,im] = df.Ranking[ix].values


In [128]:
# Create matrix M
M = np.zeros([n_users,n_movies])
# for i in range(df.shape[0]):
N = 80000
for i in range(N):
    u = df.ix[i,"User"]
    m = df.ix[i,"Movie"]
    r = df.ix[i,"Ranking"]
    M[u-1,m-1] = r
# M[M==0] = np.nan

In [179]:
# M ~= UV
k = 3
lam = float(N)/1.5
## On initialise U et V (loi uniforme)
C = 10
U =  np.random.uniform(-C,C,size=(M.shape[0],k))
V= np.random.uniform(-C,C,size=(k,M.shape[1]))

print U.shape
print V.shape


(943, 3)
(3, 1682)


In [180]:
def risque(u,v):
    s = 0
    for i in range(N):
        r = df.ix[i,"Ranking"]
        Ii = df.ix[i,"User"]-1
        Ji = df.ix[i,"Movie"]-1
        s = s + (r-u[Ii,:].dot(v[:,Ji]))**2
#         s = s + np.abs(r-u[Ii,:].dot(v[:,Ji]))
#         if i%10000==1:
#             print s
#         if np.isnan(s)==True:
#             print i
#             print u[Ii,:].dot(v[:,Ji])
#             print np.sqrt(r-u[Ii,:].dot(v[:,Ji]))
#             break
    s = float(s)/N
    return s

def test_risque(u,v):
    s = 0
    for i in range(N):
        r = df.ix[i,"Ranking"]
        Ii = df.ix[i,"User"]-1
        Ji = df.ix[i,"Movie"]-1
        s = s + (r-u[Ii,:].dot(v[:,Ji]))**2
#         s = s + np.abs(r-u[Ii,:].dot(v[:,Ji]))
#         if i%10000==1:
#             print s
#         if np.isnan(s)==True:
#             print i
#             print u[Ii,:].dot(v[:,Ji])
#             print np.sqrt(r-u[Ii,:].dot(v[:,Ji]))
#             break
    s = float(s)/N
    return s
risque(U,V)

3468.162783513877

In [182]:
## Mise a jour de u
eps = 5
# W = U.copy()
lam = 10
for iu in range(U.shape[0]):
    W = U.copy()
    W[iu,:] = W[iu,:] + np.random.uniform(-eps,eps,size=W[iu,:].shape)
#     print risque(W,V)
#     a = np.exp(-lam*risque(W,V))
#     b = np.exp(-lam*risque(U,V))
    a = lam*risque(W,V)
    b = lam*risque(U,V)
    u = np.random.rand()
    if u < b-a:
        U[iu,:] = W[iu,:]
#         print "change"
    if iu%200==0:
        print iu
        
for iv in range(V.shape[1]):
    W = V.copy()
    W[:,iv]= W[:,iv] + np.random.uniform(-eps,eps,size= W[:,iv].shape)
    a = -lam*risque(U,W)
    b = lam*risque(U,V)
    u = np.random.rand()
    if u < b-a:
        V[:,iv] = W[:,iv]
#         print "change"
    if iv%200==0:
        print iv
    
np.linalg.norm(W-U)

0
200
400
600
800
0
200
400
600
800
1000
1200
1400
1600


ValueError: operands could not be broadcast together with shapes (3,1682) (943,3) 

In [183]:
risque(U,V)

5435.700242784872

In [174]:
risque(W,V)
# np.exp(-10*risque(W,V))

18.950473312870173

In [162]:
np.exp(-180)

1.3838965267367376e-87